In [1]:
import numpy as np
def create_sequence(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), :]
        X.append(a)
        Y.append(dataset[i + look_back, :])
    return np.array(X), np.array(Y)

In [2]:
import numpy as np
import pandas as pd
# select the data
def select_data(data, start=2016, end=2024, columns=None):
    # convert the data to a pandas dataframe
    data['Date'] = pd.to_datetime(data['Date'])
    # create a mask for the data
    if columns is None:
        mask = (data['Date'].dt.year >= start) & (data['Date'].dt.year<= end)
    else:
        mask = (data['Date'].dt.year >= start) & (data['Date'].dt.year<= end) & (data.columns.isin(columns))
    # select the data
    data = data.loc[mask]
    # drop the date column
    data = data.drop(columns='Date')
    return data

In [3]:
from sklearn.preprocessing import MinMaxScaler
def create_scaler(dataset):
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)
    return scaler, dataset 

In [4]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

def create_model(dataset, look_back=1, epochs=100, batch_size=1, verbose=1):
    X, Y = create_sequence(dataset, look_back)
    X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))
    Y = np.reshape(Y, (Y.shape[0], Y.shape[1]))
    
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
    #model.add(Dropout(0.2))
    model.add(LSTM(units=50))
    model.add(Dense(X.shape[2]))  # Assuming single feature for simplicity
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X, Y, epochs=epochs, batch_size=batch_size, verbose=verbose)
    
    return model

In [13]:
def create_input_data(dataset, look_back=1):
    X = []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back), :]
        X.append(a)
    return np.array(X)

In [ ]:
# declare the path of the data
path_to_data = r"data\cleansing\filled\mean_filled.csv"

# load the data
# import pandas as pd
data = pd.read_csv(path_to_data)

col=['Rice', 'Wheat', 'Atta (Wheat)', 'Gram Dal', 'Tur/Arhar Dal']

# select the desired data
req_data = select_data(data=data, start=2023, end=2024, columns=col)

In [18]:
# scale the data
scaler, scaled_data = create_scaler(req_data) # scaled_data is a numpy array

In [19]:
# split the data into training and testing
train_size = int(len(scaled_data) * 0.80)
test_size = len(scaled_data) - train_size
train, test = scaled_data[0:train_size], scaled_data[train_size:len(scaled_data)]

In [46]:
# print with index
print("Train data: ", train[:5])

Train data:  [[0.1095994  0.         0.16772655 0.07953773 0.082198   0.24369017
  0.10357328 0.79029605 0.11842105 0.05114401 0.86484137 0.84504897
  0.79324604 0.91534991 0.98012471 0.94967287 0.57212318 0.07519313
  0.78494624 0.5524101  0.21608247 0.03897331]
 [0.1095994  0.         0.16772655 0.08701564 0.08552831 0.25645489
  0.1325738  0.79029605 0.11842105 0.05114401 0.86484137 0.84504897
  0.79324604 1.         0.98012471 0.96074484 0.55429498 0.07519313
  0.78494624 0.59908187 0.22969072 0.04511456]
 [0.1095994  0.05601318 0.15023847 0.08701564 0.082198   0.26283725
  0.12066287 0.79029605 0.11842105 0.05114401 0.89873968 0.85854189
  0.78566506 0.92179034 0.99532346 0.96074484 0.51863857 0.07519313
  0.78494624 0.60788064 0.22969072 0.0591292 ]
 [0.1095994  0.05601318 0.15023847 0.09109449 0.07372086 0.24688135
  0.10926981 0.74424342 0.11842105 0.05114401 0.89873968 0.85854189
  0.78566506 0.9887673  0.99103663 0.96074484 0.49918963 0.07519313
  0.78494624 0.54820199 0.2228

In [20]:
# create the model
model = create_model(train, look_back=1, epochs=100, batch_size=1, verbose=0)

c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
print(req_data)

       Rice  Wheat  Atta (Wheat)  Gram Dal  Tur/Arhar Dal  Urad Dal  \
2557  51.33  40.22         48.78     73.22         122.56    122.78   
2558  51.33  40.22         48.78     73.44         122.78    123.22   
2559  51.33  40.56         48.56     73.44         122.56    123.44   
2560  51.33  40.56         48.56     73.56         122.00    122.89   
2561  51.33  40.67         48.78     74.22         122.22    123.11   
...     ...    ...           ...       ...            ...       ...   
3166  56.12  44.88         52.88     99.76         172.82    137.65   
3167  57.31  45.73         54.06     99.75         173.53    139.00   
3168  57.53  45.80         54.75     99.88         172.23    138.19   
3169  57.29  45.79         55.19     99.48         172.30    138.90   
3170  57.47  45.64         53.77    100.30         173.54    139.57   

      Moong Dal  Masoor Dal  Sugar  Milk @  ...  Vanaspati (Packed)  \
2557     109.00       97.67  39.22   42.00  ...              132.11   
2558 

In [8]:
# create tthe matrix 15x4 values are in format (c+1)_(r+1)
data = np.array([['Day'+str(j+1)+'_Prod'+str(i+1) for i in range(4)] for j in range(25)])
print(data)

[['Day1_Prod1' 'Day1_Prod2' 'Day1_Prod3' 'Day1_Prod4']
 ['Day2_Prod1' 'Day2_Prod2' 'Day2_Prod3' 'Day2_Prod4']
 ['Day3_Prod1' 'Day3_Prod2' 'Day3_Prod3' 'Day3_Prod4']
 ['Day4_Prod1' 'Day4_Prod2' 'Day4_Prod3' 'Day4_Prod4']
 ['Day5_Prod1' 'Day5_Prod2' 'Day5_Prod3' 'Day5_Prod4']
 ['Day6_Prod1' 'Day6_Prod2' 'Day6_Prod3' 'Day6_Prod4']
 ['Day7_Prod1' 'Day7_Prod2' 'Day7_Prod3' 'Day7_Prod4']
 ['Day8_Prod1' 'Day8_Prod2' 'Day8_Prod3' 'Day8_Prod4']
 ['Day9_Prod1' 'Day9_Prod2' 'Day9_Prod3' 'Day9_Prod4']
 ['Day10_Prod1' 'Day10_Prod2' 'Day10_Prod3' 'Day10_Prod4']
 ['Day11_Prod1' 'Day11_Prod2' 'Day11_Prod3' 'Day11_Prod4']
 ['Day12_Prod1' 'Day12_Prod2' 'Day12_Prod3' 'Day12_Prod4']
 ['Day13_Prod1' 'Day13_Prod2' 'Day13_Prod3' 'Day13_Prod4']
 ['Day14_Prod1' 'Day14_Prod2' 'Day14_Prod3' 'Day14_Prod4']
 ['Day15_Prod1' 'Day15_Prod2' 'Day15_Prod3' 'Day15_Prod4']
 ['Day16_Prod1' 'Day16_Prod2' 'Day16_Prod3' 'Day16_Prod4']
 ['Day17_Prod1' 'Day17_Prod2' 'Day17_Prod3' 'Day17_Prod4']
 ['Day18_Prod1' 'Day18_Prod2' 'Da

In [23]:
# prepare the input data from the matrix
input_data = create_input_data(data, look_back=1)

In [15]:
print(input_data)

[[['Day1_Prod1' 'Day1_Prod2' 'Day1_Prod3' 'Day1_Prod4']]

 [['Day2_Prod1' 'Day2_Prod2' 'Day2_Prod3' 'Day2_Prod4']]

 [['Day3_Prod1' 'Day3_Prod2' 'Day3_Prod3' 'Day3_Prod4']]

 [['Day4_Prod1' 'Day4_Prod2' 'Day4_Prod3' 'Day4_Prod4']]

 [['Day5_Prod1' 'Day5_Prod2' 'Day5_Prod3' 'Day5_Prod4']]

 [['Day6_Prod1' 'Day6_Prod2' 'Day6_Prod3' 'Day6_Prod4']]

 [['Day7_Prod1' 'Day7_Prod2' 'Day7_Prod3' 'Day7_Prod4']]

 [['Day8_Prod1' 'Day8_Prod2' 'Day8_Prod3' 'Day8_Prod4']]

 [['Day9_Prod1' 'Day9_Prod2' 'Day9_Prod3' 'Day9_Prod4']]

 [['Day10_Prod1' 'Day10_Prod2' 'Day10_Prod3' 'Day10_Prod4']]

 [['Day11_Prod1' 'Day11_Prod2' 'Day11_Prod3' 'Day11_Prod4']]

 [['Day12_Prod1' 'Day12_Prod2' 'Day12_Prod3' 'Day12_Prod4']]

 [['Day13_Prod1' 'Day13_Prod2' 'Day13_Prod3' 'Day13_Prod4']]

 [['Day14_Prod1' 'Day14_Prod2' 'Day14_Prod3' 'Day14_Prod4']]

 [['Day15_Prod1' 'Day15_Prod2' 'Day15_Prod3' 'Day15_Prod4']]

 [['Day16_Prod1' 'Day16_Prod2' 'Day16_Prod3' 'Day16_Prod4']]

 [['Day17_Prod1' 'Day17_Prod2' 'Day17_Prod3'

In [16]:
print(input_data.shape)

(24, 1, 4)


In [17]:
print(input_data[-1])

[['Day24_Prod1' 'Day24_Prod2' 'Day24_Prod3' 'Day24_Prod4']]


In [21]:
print(req_data)

       Rice  Wheat  Atta (Wheat)  Gram Dal  Tur/Arhar Dal  Urad Dal  \
2557  51.33  40.22         48.78     73.22         122.56    122.78   
2558  51.33  40.22         48.78     73.44         122.78    123.22   
2559  51.33  40.56         48.56     73.44         122.56    123.44   
2560  51.33  40.56         48.56     73.56         122.00    122.89   
2561  51.33  40.67         48.78     74.22         122.22    123.11   
...     ...    ...           ...       ...            ...       ...   
3166  56.12  44.88         52.88     99.76         172.82    137.65   
3167  57.31  45.73         54.06     99.75         173.53    139.00   
3168  57.53  45.80         54.75     99.88         172.23    138.19   
3169  57.29  45.79         55.19     99.48         172.30    138.90   
3170  57.47  45.64         53.77    100.30         173.54    139.57   

      Moong Dal  Masoor Dal  Sugar  Milk @  ...  Vanaspati (Packed)  \
2557     109.00       97.67  39.22   42.00  ...              132.11   
2558 

In [30]:
print(req_data.shape)

(614, 22)


In [40]:
scaled_ip_data = scaler.transform(req_data)

In [41]:
input_data = create_input_data(scaled_ip_data, look_back=7)

In [42]:
print(input_data.shape)

(607, 7, 22)


In [43]:
pred=model.predict(input_data) 
print(pred.shape)
print(pred)

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
(607, 22)
[[ 0.21919188 -0.53827083  0.03508896 ...  1.6033721   0.7826519
  -0.22738485]
 [ 0.20783804 -0.53578055  0.03879111 ...  1.6075978   0.78898066
  -0.24597494]
 [ 0.19769655 -0.5330399   0.03630382 ...  1.6101698   0.7849747
  -0.24017982]
 ...
 [ 1.319038    0.9217061   1.2518442  ...  2.186325    1.1293279
   1.1522313 ]
 [ 1.3270621   0.92843     1.2591865  ...  2.1863961   1.1384778
   1.1329012 ]
 [ 1.3345884   0.9418945   1.2605067  ...  2.1845975   1.1286178
   1.1075441 ]]


In [44]:
# inverse the scaling
res = scaler.inverse_transform(pred)

print(res)

[[ 52.779907  36.952698  47.111416 ...  66.582146  59.258617 -14.50015 ]
 [ 52.629696  36.96781   47.15799  ...  66.69261   59.565563 -16.861279]
 [ 52.495525  36.984447  47.1267   ...  66.759834  59.371273 -16.12524 ]
 ...
 [ 67.33087   45.814754  62.418198 ...  81.82054   76.0724   160.7249  ]
 [ 67.437035  45.855568  62.510563 ...  81.822395  76.516174 158.26978 ]
 [ 67.5366    45.937298  62.527176 ...  81.77538   76.037964 155.04916 ]]


In [39]:
print(req_data.iloc[-1])

Rice                       57.470000
Wheat                      45.640000
Atta (Wheat)               53.770000
Gram Dal                  100.300000
Tur/Arhar Dal             173.540000
Urad Dal                  139.570000
Moong Dal                 122.270000
Masoor Dal                 92.410000
Sugar                      44.200000
Milk @                     47.300000
Groundnut Oil (Packed)    204.170000
Mustard Oil (Packed)      171.500000
Vanaspati (Packed)        127.370000
Soya Oil (Packed)         125.307419
Sunflower Oil (Packed)    116.400000
Palm Oil (Packed)          96.730000
Gur                        57.530000
Tea Loose                 284.170000
Salt Pack (Iodised)        24.550000
Potato                     49.900000
Onion                      56.000000
Tomato                     27.130000
Name: 3170, dtype: float64
